# I. Set up

## dependencies

In [4]:
import numpy as np
import utils
import pickle
import scipy
from sklearn import linear_model, datasets, neighbors
from sklearn.model_selection import train_test_split



%matplotlib inline
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



## Constants

In [5]:


# I pulled activations of 8 layers. Here is the index ref to the layers
LAYERS = {'pool1': 0, 'pool2': 1, 'pool3': 2, 'pool4': 3,
          'pool5': 4, 'fc6': 5, 'fc7': 6, 'prob': 7}

# constants for the images
NUM_VIEWS = 40
CARS = ['limo', 'suv', 'smart', 'sedan']
FURNITURES = ['bed', 'chair', 'table', 'bench']
OBJECTS = CARS + FURNITURES
# sample image url: https://s3.amazonaws.com/morphrecog-images-1/limoToSUV_10_10_99.png.png
# CARS_label = ['limoToSUV_10','limoToSUV_99','smartToSedan_10','smartToSedan_99'];
# FIURNITURES_label = ['bedChair_1', 'bedChair_100', 'tableBench_1', 'tableBench_100']

 

# constants for train/test for the linear discriminability
TRAIN_SZ = 0.80
TEST_SZ = 1 - TRAIN_SZ




## pulling and cleaning up the data 

In [6]:
# downloading the activations. I had to split the sample (each act has 160 images) because they're about 3GB each. 
act1 = pickle.load(open('activations.p', 'rb'))
act2 = pickle.load(open('activations2.p', 'rb'))

In [7]:
# pulling out all the fc6, fc7 layers for all images to have a smaller set to work with. 
fc6 = np.vstack((np.asarray(act1[LAYERS['fc6']]), np.asarray(act2[LAYERS['fc6']])))
fc7 = np.vstack((np.asarray(act1[LAYERS['fc7']]), np.asarray(act2[LAYERS['fc7']])))


In [78]:
# pulling out remaining pooling layers
pool = [] 

for i in xrange(0,5):
    pool1_i, pool2_i = np.asarray(act1[i]), np.asarray(act2[i])

    pool_i = np.vstack((pool1_i.reshape(pool1_i.shape[0], pool1_i.shape[1] * pool1_i.shape[2] * pool1_i.shape[3]), 
                     pool2_i.reshape(pool2_i.shape[0], pool2_i.shape[1] * pool2_i.shape[2] * pool2_i.shape[3])))
    pool.append(pool_i)

# checking that i'm pulling the arrays correctly.... 
# using pool5 because it's the smallest. 
# pool[4][20][10:50]
# foo = act1[4][20].flatten()
# foo[10:50]

# II. Representational Similarity Analysis

In [ ]:
# getMeanVector gets the mean vector across number of views for num_objects 
def getMeanVector(full_batch, num_views, num_objects):
    meanVector = np.empty((num_objects, full_batch.shape[1]), float)
    for i in xrange(0, num_objects):
        # pull out the views of the particular object 
        all_views = full_batch[i * num_views: (i + 1) * num_views, :]
        meanVector[i] = np.mean(all_views, axis = 0)

    
    return meanVector
# might need to normalize when takigng mean vector 

In [ ]:
RDM_fc7 = np.corrcoef(getMeanVector(fc7, NUM_VIEWS, len(OBJECTS))) 
imgplot = plt.imshow(RDM_fc7)
#plt.axis(['limo', 'SUV', 'smart car', 'sedan', 'bed', 'chair', 'bench', 'bed'])
# plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
plt.colorbar();
print CARS + FURNITURES

In [ ]:
RDM_fc6 = np.corrcoef(getMeanVector(fc6, NUM_VIEWS, len(OBJECTS))) 
imgplot = plt.imshow(RDM_fc6)
plt.colorbar();

In [ ]:
print "If the fc6 and fc7 RDM look similar, they are-- note:"
print "Taking the difference between fc6 and fc7, we have: "  + str((fc6 - fc7).max())
print "Taking the difference between RDM of fc6 and fc7, we have: " + str((RDM_fc6 - RDM_fc7).max())

# III. Linear Discriminability

In [ ]:

# full_batch: the full batch of images. I am assuming that the full_batch is arranged as 40 views of obj1, 40 objects of obj2... so forth.  
# train_sz: needs to be a probability, s.t. test_sz = 1 - train-sz. 
def getData(full_batch, num_views, num_objects, test_sz):
    
    num_train = int(num_views * (1 - test_sz))
    num_test = num_views - num_train
    
    X_train = np.ones((num_train * num_objects, full_batch.shape[1]), float)
    X_test = np.ones((num_test * num_objects, full_batch.shape[1]), float)
    
    for i in xrange(0, num_objects):
        X = full_batch[i * num_views: (i + 1) * num_views, :]
        
        # randomly shuffle the train/test for the x 
        X_train_i, X_test_i = train_test_split(X, test_size=test_sz)
        X_train[i * num_train: (i + 1) * num_train, :] = X_train_i
        X_test[i * num_test: (i + 1) * num_test, :] = X_test_i
        
        

    y_train = np.repeat(xrange(0, num_objects), num_train)
    y_test = np.repeat(xrange(0, num_objects), num_test)

    return (X_train, y_train, X_test, y_test)

# mini-test for the fnc 
# X_train, y_train, X_test, y_test = getData(fc6, NUM_VIEWS, len(OBJECTS), TEST_SZ)
# print "here is the full thing" 
# print np.sort(fc6[0:40,1])
# print "X_train"
# print np.sort(X_train[0:32,1])
# print "X_test"
# print np.sort(X_test[0:8, 1])


In [ ]:
logreg = linear_model.LogisticRegression()
X_train, y_train, X_test, y_test = getData(fc6, NUM_VIEWS, len(OBJECTS), TEST_SZ)
# for sanity check: X_test_r = np.random.rand(X_test.shape[0], X_test.shape[1])
print('LogisticRegression score: %f'
      % logreg.fit(X_test, y_test).score(X_train, y_train))

In [ ]:
logreg = linear_model.LogisticRegression()
X_train, y_train, X_test, y_test = getData(fc7, NUM_VIEWS, len(OBJECTS), TEST_SZ)
# for sanity check: X_test_r = np.random.rand(X_test.shape[0], X_test.shape[1])
print('LogisticRegression score: %f'
      % logreg.fit(X_test, y_test).score(X_train, y_train))